In [1]:
import torch
import torchvision

from avalanche.models import IncrementalClassifier, MultiHeadClassifier, MultiTaskModule
import os
os.chdir("../")
from Scaled_KAN import *
os.chdir("Split_MNIST")
from avalanche.benchmarks.datasets import MNIST
from avalanche.benchmarks.datasets.torchvision_wrapper import CIFAR100

from avalanche.benchmarks.datasets.dataset_utils import default_dataset_location
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.benchmarks.utils import as_classification_dataset, AvalancheDataset

from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.supervised import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC  # and many more!
from avalanche.training.templates import SupervisedTemplate
from avalanche.training.plugins import EvaluationPlugin, ReplayPlugin, EWCPlugin

from avalanche.logging import InteractiveLogger

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import SplitMNIST, SplitCIFAR100
from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics

# from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_with_validation_stream

device = "cuda" if torch.cuda.is_available() else "cpu"

################################
# DEFINE THE MODELS TO BE USED #
################################

IN_DIM = 32*32
model_KAN = FastKAN(layers_hidden=[IN_DIM,32,10], num_grids=4, device=device)
model_incremental = IncrementalClassifier(in_features=IN_DIM)
model_multihead = MultiHeadClassifier(in_features=IN_DIM)
# model_multi_task = MultiTaskModule(in_features=IN_DIM)

################################
# DOWNLOAD DATASETS TO BE USED #
################################

# datadir = default_dataset_location('mnist')
# train_MNIST = as_classification_dataset(MNIST(datadir, train=True, download=True))
# test_MNIST = as_classification_dataset(MNIST(datadir, train=False, download=True))

# datadir = default_dataset_location('cifar100')
# train_CIFAR100 = as_classification_dataset(CIFAR100(datadir, train=True, download=True))
# test_CIFAR100 = as_classification_dataset(CIFAR100(datadir, train=False, download=True))


# ################################
#     CREATE THE BENCHMARK     #
################################

# benchmark = benchmark_from_datasets(
#     train=[train_MNIST, train_CIFAR100],
#     test=[test_MNIST, test_CIFAR100]
# )

benchmark = SplitCIFAR100(
    n_experiences=50,  # 5 incremental experiences
    return_task_id=True,  # add task labels
    seed=1  # you can set the seed for reproducibility. This will fix the order of classes
)


print(f"{benchmark.train_stream.name} - len {len(benchmark.train_stream)}")
print(f"{benchmark.test_stream.name} - len {len(benchmark.test_stream)}")
print(f"{benchmark._streams}")

################################
#    DEFINE THE OPTIMISER      #
################################
optimizer_KAN = SGD(model_KAN.parameters(), lr=0.001, momentum=0.9)
optimizer_incremental = SGD(model_incremental.parameters(), lr=0.001, momentum=0.9)
optimizer_multihead = SGD(model_multihead.parameters(), lr=0.001, momentum=0.9)

################################
#       DEFINE THE LOSS        #
################################
criterion_KAN = CrossEntropyLoss()
criterion_incremental = CrossEntropyLoss()
criterion_multihead = CrossEntropyLoss()

################################
# DEFINE THE EVALUATION PLUGIN #
################################
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[InteractiveLogger()],
    strict_checks=False
)

################################
#      CREATE THE STRATEGY     #
################################
cl_strategy_KAN = Naive(
    model=model_KAN, optimizer=optimizer_KAN, criterion=criterion_KAN,
    train_mb_size=100, train_epochs=4, eval_mb_size=100, evaluator=eval_plugin
)
cl_strategy_incremental = Naive(
    model=model_incremental, optimizer=optimizer_incremental, criterion=criterion_incremental,
    train_mb_size=100, train_epochs=4, eval_mb_size=100, evaluator=eval_plugin
)
cl_strategy_multihead = Naive(
    model=model_multihead, optimizer=optimizer_multihead, criterion=criterion_multihead,
    train_mb_size=100, train_epochs=4, eval_mb_size=100, evaluator=eval_plugin
)

################################
#        TRAINING LOOP         #
################################

print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    # train returns a dictionary which contains all the metric values
    print(f"Experience {experience}")
    res = cl_strategy_KAN.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_KAN.eval(benchmark.test_stream))


/home/thealch3mist/University/Thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified
train - len 50
test - len 50
{'train': <avalanche.benchmarks.scenarios.deprecated.new_classes.nc_scenario.NCStream object at 0x7c68561c10a0>, 'test': <avalanche.benchmarks.scenarios.deprecated.new_classes.nc_scenario.NCStream object at 0x7c68561c1070>}
Starting experiment...
Experience <avalanche.benchmarks.scenarios.deprecated.new_classes.nc_scenario.NCExperience object at 0x7c68561c2e10>
-- >> Start of training phase << --
0it [00:00, ?it/s]

RuntimeError: Given normalized_shape=[1024], expected input with shape [*, 1024], but got input of size[100, 3, 32, 32]

In [2]:
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_with_validation_stream


print(f"original training samples = {len(benchmark.train_stream[0].dataset)}")

benchmark = benchmark_with_validation_stream(benchmark, validation_size=0.25)
print(f"new training samples = {len(benchmark.train_stream[0].dataset)}")
print(f"validation samples = {len(benchmark.valid_stream[0].dataset)}")

ImportError: cannot import name 'benchmark_with_validation_stream' from 'avalanche.benchmarks.scenarios.dataset_scenario' (/home/thealch3mist/University/Thesis/venv/lib/python3.12/site-packages/avalanche/benchmarks/scenarios/dataset_scenario.py)

In [43]:
from avalanche.benchmarks.classic import SplitMNIST

benchmark1 = SplitMNIST(
    n_experiences=5,
    return_task_id=True,  # add task labels
    seed=1
)

print(f'--- Stream: {benchmark1.train_stream.name}')

for exp in benchmark1.train_stream:
    eid = exp.current_experience # experience id
    clss = exp.classes_in_this_experience # classes in this experience
    tls = exp.task_labels # task labels
    print(f"EID={eid}, classes={clss}, tasks={tls}")
    print(f"data: {len(exp.dataset)} samples")



--- Stream: train
EID=0, classes=[5, 6], tasks=[0]
data: 11339 samples
EID=1, classes=[1, 2], tasks=[1]
data: 12700 samples
EID=2, classes=[0, 8], tasks=[2]
data: 11774 samples
EID=3, classes=[9, 3], tasks=[3]
data: 12080 samples
EID=4, classes=[4, 7], tasks=[4]
data: 12107 samples


In [1]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.supervised import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC  # and many more!
from avalanche.training.templates import SupervisedTemplate
from avalanche.training.plugins import EvaluationPlugin, ReplayPlugin, EWCPlugin

from avalanche.logging import InteractiveLogger

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP

device = "cuda" if torch.cuda.is_available() else "cpu"

################################
# DEFINE THE MODELS TO BE USED #
################################
model_KAN = FastKAN(layers_hidden=[28*28,32,10], num_grids=4, device=device)
model_incremental = IncrementalClassifier(in_features=28*28)
model_multihead = MultiHeadClassifier(in_features=28*28)

################################
#    DEFINE THE OPTIMISER      #
################################
optimizer_KAN = SGD(model_KAN.parameters(), lr=0.001, momentum=0.9)
optimizer_incremental = SGD(model_incremental.parameters(), lr=0.001, momentum=0.9)
optimizer_multihead = SGD(model_multihead.parameters(), lr=0.001, momentum=0.9)

################################
#       DEFINE THE LOSS        #
################################
criterion_KAN = CrossEntropyLoss()
criterion_incremental = CrossEntropyLoss()
criterion_multihead = CrossEntropyLoss()

################################
# DEFINE THE EVALUATION PLUGIN #
################################
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[InteractiveLogger()],
    strict_checks=False
)

################################
#      CREATE THE STRATEGY     #
################################
cl_strategy_KAN = Naive(
    model_KAN, optimizer_KAN, criterion_KAN,
    train_mb_size=100, train_epochs=4, eval_mb_size=100, evaluator=eval_plugin
)
cl_strategy_incremental = Naive(
    model_incremental, optimizer_incremental, criterion_incremental,
    train_mb_size=100, train_epochs=4, eval_mb_size=100, evaluator=eval_plugin
)
cl_strategy_multihead = Naive(
    model_multihead, optimizer_multihead, criterion_multihead,
    train_mb_size=100, train_epochs=4, eval_mb_size=100, evaluator=eval_plugin
)

################################
#        TRAINING LOOP         #
################################

print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    # train returns a dictionary which contains all the metric values
    res = cl_strategy_KAN.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_KAN.eval(benchmark.test_stream))


/home/thealch3mist/University/Thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'torch' is not defined

In [3]:
from avalanche.benchmarks.classic import SplitMNIST

# scenario
benchmark = SplitMNIST(n_experiences=5, seed=1)

# TRAINING LOOP
print('Starting experiment...')
results_KAN = []
results_incremental = []
results_multihead = []

for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy_KAN.train(experience)
    cl_strategy_incremental.train(experience)
    cl_strategy_multihead.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results_KAN.append(cl_strategy_KAN.eval(benchmark.test_stream))
    results_incremental.append(cl_strategy_incremental.eval(benchmark.test_stream))
    results_multihead.append(cl_strategy_multihead.eval(benchmark.test_stream))


Starting experiment...
Start of experience:  0
Current Classes:  [5, 6]


NameError: name 'cl_strategy_KAN' is not defined

In [36]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.supervised import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC  # and many more!

model = SimpleMLP(num_classes=10)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()
cl_strategy = Naive(
    model, optimizer, criterion,
    train_mb_size=100, train_epochs=4, eval_mb_size=100
)
from avalanche.benchmarks.classic import SplitMNIST

# scenario
benchmark = SplitMNIST(n_experiences=5, seed=1)

# TRAINING LOOP
# print('Starting experiment...')
results = [] 
for experience in benchmark.train_stream:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience)
    # print('Training completed')

    # print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(benchmark.test_stream))

/home/thealch3mist/University/Thesis/venv/lib/python3.12/site-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


-- >> Start of training phase << --
100%|██████████| 114/114 [00:01<00:00, 61.15it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4083
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8932
100%|██████████| 114/114 [00:01<00:00, 59.17it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1044
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9692
100%|██████████| 114/114 [00:01<00:00, 59.15it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0837
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9742
100%|██████████| 114/114 [00:01<00:00, 57.87it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0753
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9763
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 69.28it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/tes

In [35]:

for i in results:
    print(i)

{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9744245524296675, 'Loss_Epoch/train_phase/train_stream/Task000': 0.07835279670509883, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9816216216216216, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.06574291653452895, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.0, 'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 7.920903306212027, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.0, 'Loss_Exp/eval_phase/test_stream/Task000/Exp002': 10.382590512536321, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003': 0.0, 'Loss_Exp/eval_phase/test_stream/Task000/Exp003': 10.082500989629585, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004': 0.0, 'Loss_Exp/eval_phase/test_stream/Task000/Exp004': 8.420142700423055, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.1816, 'Loss_Stream/eval_phase/test_stream/Task000': 7.485486004755879}
{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9878740157480315, 'Loss_